In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from autokeras.keras_layers import ExpandLastDim
from autokeras.keras_layers import CastToFloat32
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, LSTM, SimpleRNN
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers.experimental.preprocessing import RandomTranslation, RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.applications import EfficientNetB7
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Make Input

In [3]:
def calculateWeight(tlx):
    tlx_weight = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Sum':[0]}
    tlx_weight = pd.DataFrame(tlx_weight)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col1 in range(1,len(tlx.columns)):
            for col2 in range(col1+1, len(tlx.columns)):
                if tlx[tlx.columns[col1]][i] > tlx[tlx.columns[col2]][i]:
                    score[col1-1]+=1
                elif tlx[tlx.columns[col1]][i] < tlx[tlx.columns[col2]][i]:
                    score[col2-1]+=1
                else :
                    score[col1-1]+=0.5
                    score[col2-1]+=0.5
                    
        score[6] = score[0]+score[1]+score[2]+score[3]+score[4]+score[5]
        tlx_weight.loc[i]=score
    #print(tlx_weight.loc[0])
    return tlx_weight

def calculate_tlxLevel(tlx, tlx_weight):
    result = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Score':[0]}
    result = pd.DataFrame(result)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col in range(len(tlx_weight.columns)-1):
            score[col] = int(tlx[tlx.columns[col+1]].loc[i] * tlx_weight[tlx_weight.columns[col]].loc[i] )
        score[6] =int((score[0]+score[1]+score[2]+score[3]+score[4]+score[5] )/ tlx_weight[tlx_weight.columns[6]].loc[i]/10 + 0.5)
        if score[6]>10: score[6]=10
        if score[6]<0: score[6]=0
        result.loc[i]=score
    return result['Score']

In [10]:
def split_data(df, label, end):
    col = ['EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7',
           'EEG.P7', 'EEG.O1', 'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6', 'EEG.F4',
           'EEG.F8', 'EEG.AF4', 'MarkerValueInt']
    col_rename = ['AF3', 'F7', 'F3', 'FC5', 'T7',
           'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4',
           'F8', 'AF4', 'vis_name']
    data_extraction = df[col]
    data_extraction.columns = col_rename
    rest = data_extraction[data_extraction.vis_name == 0]
    survey = data_extraction[data_extraction.vis_name == 100]
    
    vis = data_extraction[data_extraction.vis_name == 1].reset_index(drop=True)
    vis['label'] = label[0]
    vis.drop(['vis_name'], axis=1, inplace=True)
    
    for i in range(2,end):
        df = data_extraction[data_extraction.vis_name == i].reset_index(drop=True)
        df['label'] = label[i-1]
        df.drop(['vis_name'], axis=1, inplace=True)
        vis = pd.concat([vis,df], ignore_index=True, axis=0)
    return rest, survey, vis

In [23]:
def split_fft_data(df, label, end):
    col = ['POW.AF3.Theta', 'POW.AF3.Alpha',
       'POW.AF3.BetaL', 'POW.AF3.BetaH', 'POW.AF3.Gamma', 'POW.F7.Theta',
       'POW.F7.Alpha', 'POW.F7.BetaL', 'POW.F7.BetaH', 'POW.F7.Gamma',
       'POW.F3.Theta', 'POW.F3.Alpha', 'POW.F3.BetaL', 'POW.F3.BetaH',
       'POW.F3.Gamma', 'POW.FC5.Theta', 'POW.FC5.Alpha', 'POW.FC5.BetaL',
       'POW.FC5.BetaH', 'POW.FC5.Gamma', 'POW.T7.Theta', 'POW.T7.Alpha',
       'POW.T7.BetaL', 'POW.T7.BetaH', 'POW.T7.Gamma', 'POW.P7.Theta',
       'POW.P7.Alpha', 'POW.P7.BetaL', 'POW.P7.BetaH', 'POW.P7.Gamma',
       'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O1.BetaL', 'POW.O1.BetaH',
       'POW.O1.Gamma', 'POW.O2.Theta', 'POW.O2.Alpha', 'POW.O2.BetaL',
       'POW.O2.BetaH', 'POW.O2.Gamma', 'POW.P8.Theta', 'POW.P8.Alpha',
       'POW.P8.BetaL', 'POW.P8.BetaH', 'POW.P8.Gamma', 'POW.T8.Theta',
       'POW.T8.Alpha', 'POW.T8.BetaL', 'POW.T8.BetaH', 'POW.T8.Gamma',
       'POW.FC6.Theta', 'POW.FC6.Alpha', 'POW.FC6.BetaL', 'POW.FC6.BetaH',
       'POW.FC6.Gamma', 'POW.F4.Theta', 'POW.F4.Alpha', 'POW.F4.BetaL',
       'POW.F4.BetaH', 'POW.F4.Gamma', 'POW.F8.Theta', 'POW.F8.Alpha',
       'POW.F8.BetaL', 'POW.F8.BetaH', 'POW.F8.Gamma', 'POW.AF4.Theta',
       'POW.AF4.Alpha', 'POW.AF4.BetaL', 'POW.AF4.BetaH', 'POW.AF4.Gamma', 'MarkerValueInt']
    col_rename = ['AF3.Theta', 'AF3.Alpha','AF3.BetaL', 'AF3.BetaH', 'AF3.Gamma', 
                  'F7.Theta','F7.Alpha', 'F7.BetaL', 'F7.BetaH', 'F7.Gamma',
                  'F3.Theta', 'F3.Alpha', 'F3.BetaL', 'F3.BetaH','F3.Gamma', 
                  'FC5.Theta', 'FC5.Alpha', 'FC5.BetaL','FC5.BetaH', 'FC5.Gamma', 
                  'T7.Theta', 'T7.Alpha', 'T7.BetaL', 'T7.BetaH', 'T7.Gamma', 
                  'P7.Theta', 'P7.Alpha', 'P7.BetaL', 'P7.BetaH', 'P7.Gamma',
                  'O1.Theta', 'O1.Alpha', 'O1.BetaL', 'O1.BetaH','O1.Gamma',
                  'O2.Theta', 'O2.Alpha', 'O2.BetaL','O2.BetaH', 'O2.Gamma',
                  'P8.Theta', 'P8.Alpha', 'P8.BetaL', 'P8.BetaH', 'P8.Gamma', 
                  'T8.Theta','T8.Alpha', 'T8.BetaL', 'T8.BetaH', 'T8.Gamma',
                  'FC6.Theta', 'FC6.Alpha', 'FC6.BetaL', 'FC6.BetaH','FC6.Gamma', 
                  'F4.Theta', 'F4.Alpha', 'F4.BetaL','F4.BetaH', 'F4.Gamma',
                  'F8.Theta', 'F8.Alpha','F8.BetaL', 'F8.BetaH', 'F8.Gamma',
                  'AF4.Theta', 'AF4.Alpha', 'AF4.BetaL', 'AF4.BetaH', 'AF4.Gamma', 'vis_name']
    data_extraction = df[col]
    data_extraction.columns = col_rename
    rest = data_extraction[data_extraction.vis_name == 0].dropna(axis=0)
    survey = data_extraction[data_extraction.vis_name == 100].dropna(axis=0)
    
    vis = data_extraction[data_extraction.vis_name == 1].reset_index(drop=True).dropna(axis=0)
    vis['label'] = label[0]
    vis.drop(['vis_name'], axis=1, inplace=True)
    
    for i in range(2,end):
        df = data_extraction[data_extraction.vis_name == i].reset_index(drop=True)
        df['label'] = label[i-1]
        df.drop(['vis_name'], axis=1, inplace=True)
        df = df.dropna(axis=0)
        vis = pd.concat([vis,df], ignore_index=True, axis=0)
    return rest, survey, vis

In [ ]:
def cal_theta_alpha(df, labe, end):
    col = ['POW.AF3.Theta', 'POW.AF3.Alpha', 'POW.F7.Theta', 'POW.F7.Alpha', 
           'POW.F3.Theta', 'POW.F3.Alpha', 'POW.FC5.Theta', 'POW.FC5.Alpha', 
           'POW.T7.Theta', 'POW.T7.Alpha', 'POW.P7.Theta', 'POW.P7.Alpha',
           'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O2.Theta', 'POW.O2.Alpha', 
           'POW.P8.Theta', 'POW.P8.Alpha', 'POW.T8.Theta', 'POW.T8.Alpha',
           'POW.FC6.Theta', 'POW.FC6.Alpha',  'POW.F4.Theta', 'POW.F4.Alpha', 
           'POW.F8.Theta', 'POW.F8.Alpha','POW.AF4.Theta', 'POW.AF4.Alpha','MarkerValueInt']
    col_rename = ['AF3.Theta', 'AF3.Alpha', 'F7.Theta','F7.Alpha', 
                  'F3.Theta', 'F3.Alpha', 'FC5.Theta', 'FC5.Alpha', 
                  'T7.Theta', 'T7.Alpha', 'P7.Theta', 'P7.Alpha', 
                  'O1.Theta', 'O1.Alpha', 'O2.Theta', 'O2.Alpha',
                  'P8.Theta', 'P8.Alpha', 'T8.Theta','T8.Alpha',
                  'FC6.Theta', 'FC6.Alpha', 'F4.Theta', 'F4.Alpha', 
                  'F8.Theta', 'F8.Alpha', 'AF4.Theta', 'AF4.Alpha','vis_name']
    col2 = ['AF3', , 'F7','F3', 'FC5', 'T7', 'P7', 'O1', 'O2',
            'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4','vis_name']

def split_theta_alpha_data(df, label, end):
    col = ['POW.AF3.Theta', 'POW.AF3.Alpha', 'POW.F7.Theta', 'POW.F7.Alpha', 
           'POW.F3.Theta', 'POW.F3.Alpha', 'POW.FC5.Theta', 'POW.FC5.Alpha', 
           'POW.T7.Theta', 'POW.T7.Alpha', 'POW.P7.Theta', 'POW.P7.Alpha',
           'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O2.Theta', 'POW.O2.Alpha', 
           'POW.P8.Theta', 'POW.P8.Alpha', 'POW.T8.Theta', 'POW.T8.Alpha',
           'POW.FC6.Theta', 'POW.FC6.Alpha',  'POW.F4.Theta', 'POW.F4.Alpha', 
           'POW.F8.Theta', 'POW.F8.Alpha','POW.AF4.Theta', 'POW.AF4.Alpha','MarkerValueInt']
    col_rename = ['AF3.Theta', 'AF3.Alpha', 'F7.Theta','F7.Alpha', 
                  'F3.Theta', 'F3.Alpha', 'FC5.Theta', 'FC5.Alpha', 
                  'T7.Theta', 'T7.Alpha', 'P7.Theta', 'P7.Alpha', 
                  'O1.Theta', 'O1.Alpha', 'O2.Theta', 'O2.Alpha',
                  'P8.Theta', 'P8.Alpha', 'T8.Theta','T8.Alpha',
                  'FC6.Theta', 'FC6.Alpha', 'F4.Theta', 'F4.Alpha', 
                  'F8.Theta', 'F8.Alpha', 'AF4.Theta', 'AF4.Alpha','vis_name']
    col2 = ['AF3', , 'F7','F3', 'FC5', 'T7', 'P7', 'O1', 'O2',
            'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4','vis_name']
    data_extraction = df[col]
    data_extraction.columns = col_rename
    rest = data_extraction[data_extraction.vis_name == 0].dropna(axis=0)
    survey = data_extraction[data_extraction.vis_name == 100].dropna(axis=0)
    
    vis = data_extraction[data_extraction.vis_name == 1].reset_index(drop=True).dropna(axis=0)
    vis['label'] = label[0]
    vis.drop(['vis_name'], axis=1, inplace=True)
    
    for i in range(2,end):
        df = data_extraction[data_extraction.vis_name == i].reset_index(drop=True)
        df['label'] = label[i-1]
        df.drop(['vis_name'], axis=1, inplace=True)
        df = df.dropna(axis=0)
        vis = pd.concat([vis,df], ignore_index=True, axis=0)
    return rest, survey, vis

In [5]:
src = 'EEG data/User/'
files = os.listdir(src)
tlx=[]
for f in files:
    tlx.append(pd.read_csv(src+f))
src = 'EEG data/prepross/'
datas = os.listdir(src)
workloadLevel = []
for t in tlx:
    workloadLevel.append(calculate_tlxLevel(t, calculateWeight(t)))
eegData=[]
for d in datas:
    eegData.append(pd.read_csv(src+d))

In [12]:
cnt = 0
for eeg, label in zip (eegData, workloadLevel):
    if cnt == 0 :
        rest, survey, vis = split_data(eeg, label,21)
        cnt+=1
        continue
    elif cnt == 6:
        r, s, v = split_data(eeg, label,21)
    else:
        r, s, v = split_data(eeg, label, 22)
    
    rest = pd.concat([rest,r], ignore_index=True, axis=0)
    survey = pd.concat([survey,s], ignore_index=True, axis=0)
    vis = pd.concat([vis,v], ignore_index=True, axis=0)
    cnt+=1

In [24]:
cnt = 0
for eeg, label in zip (eegData, workloadLevel):
    if cnt == 0 :
        rest_fft, survey_fft, vis_fft = split_fft_data(eeg, label,21)
        cnt+=1
        continue
    elif cnt == 6:
        r_fft, s_fft, v_fft = split_fft_data(eeg, label,21)
    else:
        r_fft, s_fft, v_fft = split_fft_data(eeg, label, 22)
    
    rest_fft = pd.concat([rest_fft,r_fft], ignore_index=True, axis=0)
    survey_fft = pd.concat([survey_fft,s_fft], ignore_index=True, axis=0)
    vis_fft = pd.concat([vis_fft,v_fft], ignore_index=True, axis=0)
    cnt+=1

In [25]:
vis_fft

,AF3.Theta,AF3.Alpha,AF3.BetaL,AF3.BetaH,AF3.Gamma,F7.Theta,F7.Alpha,F7.BetaL,F7.BetaH,F7.Gamma,...,F8.Alpha,F8.BetaL,F8.BetaH,F8.Gamma,AF4.Theta,AF4.Alpha,AF4.BetaL,AF4.BetaH,AF4.Gamma,label
0,9.173274,3.106601,1.099620,1.028382,0.398435,13.847454,4.459079,2.059237,1.885073,0.794723,...,2.298744,1.411010,0.934551,0.511227,7.413913,2.932344,1.782573,1.104984,0.438779,6
1,10.970263,3.286717,0.980125,1.106689,0.399811,15.437139,5.031683,2.356469,2.520724,0.832195,...,2.435528,1.795794,1.151114,0.529097,8.105520,3.309745,2.431626,1.475698,0.488555,6
2,11.896807,3.438130,0.962297,1.164912,0.420656,15.781597,5.461060,2.844015,3.333000,0.904789,...,2.668025,2.300908,1.370913,0.558700,8.082989,3.821387,3.237942,1.950380,0.566002,6
3,11.761883,3.459307,1.037847,1.206656,0.465319,15.123814,5.583092,3.381478,4.166084,1.007141,...,2.909856,2.793795,1.569230,0.599309,7.542512,4.307007,3.964213,2.427696,0.656156,6
4,10.779994,3.326084,1.178284,1.237358,0.524978,13.572821,5.360179,3.808501,4.792826,1.111347,...,3.096153,3.122208,1.724803,0.639429,6.603131,4.609829,4.365841,2.768109,0.730073,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108207,16.052397,4.474069,0.948681,0.759617,0.474118,10.951871,2.513503,0.433273,0.292269,0.184488,...,2.237617,0.881767,0.169846,0.135366,15.739989,3.450108,1.418323,0.448079,0.483714,5
108208,19.973568,5.400219,0.879941,0.789202,0.493196,12.166910,2.480561,0.431624,0.281479,0.176217,...,2.060185,0.678499,0.163792,0.132977,18.980492,3.998825,1.359801,0.512423,0.443753,5
108209,21.391216,5.843330,0.768417,0.776957,0.497462,12.406390,2.315039,0.425017,0.273872,0.173656,...,1.845860,0.476338,0.166581,0.139204,20.895807,4.281593,1.181604,0.586123,0.418840,5
108210,19.844902,5.645937,0.656863,0.721887,0.486074,12.232596,2.089117,0.438556,0.281801,0.183639,...,1.706487,0.343786,0.178487,0.154436,20.853188,4.226102,0.942141,0.645811,0.403860,5


In [7]:
label=vis['label']
label = to_categorical(label,11)
data=vis.drop(['label'],axis=1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
data = pd.DataFrame(scaled, columns = data.columns, index=data.index)
data
print(label.shape)

(1731369, 11)


In [42]:
label_fft=vis_fft['label']
label_fft = to_categorical(label_fft,11)
data_fft=vis_fft.drop(['label'],axis=1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data_fft)
data_fft = pd.DataFrame(scaled, columns = data_fft.columns, index=data_fft.index)
data_fft
print(data_fft.shape, label_fft.shape)

(108212, 70) (108212, 11)


In [51]:
def windowing_dataset(data, label, window_size):
    data_list = []
    label_list = []
    
    for i in range(0,len(data)//window_size):
        data_list.append(np.array(data.iloc[i*window_size:(i+1)*window_size]))
        label_list.append(np.array(label.iloc[i*window_size]))
    return np.array(data_list), np.array(label_list)

In [33]:
x, y = windowing_dataset(data,pd.DataFrame(label),10)
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = y)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(13, 70, 70) (10, 70, 70) (13, 11) (10, 11)


In [11]:
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

def drawResult(history):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper left')

    plt.show()
    
def runModel(model, crossentropy, x_train, y_train, x_valid, y_valid):
    early_stop = EarlyStopping(monitor='val_loss', patience=10)
    model.compile(optimizer='adam', loss= crossentropy, metrics='accuracy')

    history = model.fit(x_train, y_train, 
                        epochs = 200, 
                        validation_data = (x_valid, y_valid), 
                        callbacks=[early_stop], verbose=1)
    drawResult(history)
    return model

def eval_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    return f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')

In [10]:
def Model2_2_1():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_2_2():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(11, activation='sigmoid'))
    
    model.summary()
    return model

def Model2_2_3():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

crossentropy = 'categorical_crossentropy'

print(eval_model(runModel(Model2_2_1(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_2_2(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_2_3(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))

In [ ]:
def Model2_1_1():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_1_2():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_1_3():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

crossentropy = 'categorical_crossentropy'

print(eval_model(runModel(Model2_1_1(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_1_2(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_1_3(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))

# AUTO KERAS

In [29]:
import autokeras as ak
from tensorflow.keras.utils import plot_model
from datetime import datetime

In [52]:
x_fft, y_fft = windowing_dataset(data_fft,pd.DataFrame(label_fft),10)
print(x_fft.shape, y_fft.shape)
x_train, x_test, y_train, y_test = train_test_split(x_fft,y_fft, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = y_fft)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(10821, 10, 70) (10821, 11)
(6492, 10, 70) (4329, 10, 70) (6492, 11) (4329, 11)


In [53]:
trials=[3,5,10,20]
for trial in trials:
    clf_ = ak.ImageClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=x_train, y=y_train, epochs=20)

    predicted_y = clf_.predict(x_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(x_test, y_test)
    print(clf_.evaluate(x_test, y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/'+'_ACC_'+str(tmp)+'_try_'+str(trial)+'_.h5')

Trial 2 Complete [00h 12m 36s]
val_loss: 1.878928780555725

Best val_loss So Far: 1.2487961053848267
Total elapsed time: 00h 12m 55s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|efficient         |vanilla           
image_block_1/n...|True              |True              
image_block_1/a...|True              |False             
image_block_1/i...|True              |None              
image_block_1/i...|False             |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0.1               |None              
image_block_1/i...|0                 |None              
image_block_1/e...|True              |None              
image_block_1/e...|b7                |None              
image_block_1/e...|True              |None              
image_block_1/e...|True              |None              
classification_...|global_avg        |flat

KeyboardInterrupt: 

In [54]:
x_train, x_test, y_train, y_test = train_test_split(data_fft,label_fft, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = label_fft)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(64927, 70) (43285, 70) (64927, 11) (43285, 11)


In [56]:
data_fft.dtypes

AF3.Theta    float64
AF3.Alpha    float64
AF3.BetaL    float64
AF3.BetaH    float64
AF3.Gamma    float64
              ...   
AF4.Theta    float64
AF4.Alpha    float64
AF4.BetaL    float64
AF4.BetaH    float64
AF4.Gamma    float64
Length: 70, dtype: object

In [59]:
trials=[20]
for trial in trials:
    clf_ = ak.StructuredDataClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=x_train, y=y_train, epochs=20)

    predicted_y = clf_.predict(x_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(x_test, y_test)
    print(clf_.evaluate(x_test, y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/'+'S_ACC_'+str(tmp)+'_try_'+str(trial),save_format='tf')

Trial 10 Complete [00h 07m 21s]
val_accuracy: 0.6475808024406433

Best val_accuracy So Far: 0.6685701012611389
Total elapsed time: 01h 05m 40s
Epoch 1/20
2029/2029 [==============================] - 20s 9ms/step - loss: 1.4216 - accuracy: 0.4397
Epoch 2/20
2029/2029 [==============================] - 20s 10ms/step - loss: 1.1745 - accuracy: 0.5235
Epoch 3/20
2029/2029 [==============================] - 21s 10ms/step - loss: 1.0810 - accuracy: 0.5576
Epoch 4/20
2029/2029 [==============================] - 19s 10ms/step - loss: 1.0199 - accuracy: 0.5834
Epoch 5/20
2029/2029 [==============================] - 19s 10ms/step - loss: 0.9674 - accuracy: 0.6051
Epoch 6/20
2029/2029 [==============================] - 19s 9ms/step - loss: 0.9239 - accuracy: 0.6230
Epoch 7/20
2029/2029 [==============================] - 19s 9ms/step - loss: 0.8881 - accuracy: 0.6388
Epoch 8/20
2029/2029 [==============================] - 19s 9ms/step - loss: 0.8551 - accuracy: 0.6534
Epoch 9/20
2029/2029 [=======

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer MultiCategoryEncoding with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002337B835B08>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x00000233794D7608>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x0000023391BEF308>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002337B6C50C8>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002337B6C5CC8>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002337B58E8C8>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000002337B60C6C8>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object a

In [60]:
model.save('model/'+'S_ACC_'+str(tmp)+'_try_'+str(trial),save_format='tf')

1D CNN? / windowing LSTM?